# Feature Resolution and Residuals Experiment (WIP)

This notebook uses the SageWorks Framework to quickly build an AWS® Machine Learning Pipeline with the AQSolDB public dataset. For this exercise we're going to look at the relationship between feature space and target values, specifically we're going to use SageWorks to help us identify areas where compounds that are close in feature space have significant differences in their target values (solubility in this case).


## Data
AqSolDB: A curated reference set of aqueous solubility, created by the Autonomous Energy Materials Discovery [AMD] research group, consists of aqueous solubility values of 9,982 unique compounds curated from 9 different publicly available aqueous solubility datasets. AqSolDB also contains some relevant topological and physico-chemical 2D descriptors. Additionally, AqSolDB contains validated molecular representations of each of the compounds. This openly accessible dataset, which is the largest of its kind, and will not only serve as a useful reference source of measured and calculated solubility data, but also as a much improved and generalizable training data source for building data-driven models. (2019-04-10)

Main Reference:
https://www.nature.com/articles/s41597-019-0151-1

Data Dowloaded from the Harvard DataVerse:
https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/OVHAW8

® Amazon Web Services, AWS, the Powered by AWS logo, are trademarks of Amazon.com, Inc. or its affiliates.

In [ ]:
import sageworks
import logging
logging.getLogger("sageworks").setLevel(logging.WARNING)

In [ ]:
# We've already created a FeatureSet so just grab it a sample
from sageworks.api import FeatureSet
fs = FeatureSet("test_sol_nightly_log_s")
full_df = fs.pull_dataframe()

In [ ]:
full_df.head()

In [ ]:
# Sanity check our solubility and solubility_class
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 12.0
plt.rcParams['figure.figsize'] = 14.0, 5.0
sns.set_theme(style='darkgrid')

# Create a box plot
ax = sns.boxplot(x='class', y='log_s', data=full_df, order = [2, 1, 0])
plt.title('Solubility by Solubility Class')
plt.xlabel('Solubility Class')
plt.ylabel('Solubility')
plt.show()

In [ ]:
Chem.MolFromSmiles("CCC(=O)OC(CC1=CC=CC=C1)(C(C)CN(C)C)C2=CC=CC=C2")

In [ ]:
Chem.MolFromSmiles("CCC(=O)O[C@@](Cc1ccccc1)([C@H](C)CN(C)C)c2ccccc2")

In [ ]:
def show(id):
    smile = df[df["id"]==id]["smiles"].values[0]
    print(smile)
    _features = df[df["id"]==id][features].values[0]
    print(_features)
    _target = df[df["id"]==id][target].values[0]
    print(_target)
    return Chem.MolFromSmiles(smile)

In [ ]:
close_ids = ["E-1200", "A-3473", "B-1665"]
show("E-1200")

In [ ]:
show("A-3473")

In [ ]:
show("B-1665")

In [ ]:
from rdkit import Chem

# Create an RDKit molecule from a SMILES string
smiles = "CCC(=O)OC(CC1=CC=CC=C1)(C(C)CN(C)C)C2=CC=CC=C2"
mol = Chem.MolFromSmiles(smiles)

# Assign stereochemistry using RDKit
Chem.AssignStereochemistry(mol, cleanIt=True, force=True)

# Find chiral centers and their configurations
chiral_centers = Chem.FindMolChiralCenters(mol, includeUnassigned=True)

# Print the results
for center in chiral_centers:
    index, configuration = center
    print(f"Atom index: {index}, Configuration: {configuration}")

In [ ]:
# Create an RDKit molecule from a SMILES string
smiles = "CCC(=O)O[C@@](Cc1ccccc1)([C@H](C)CN(C)C)c2ccccc2"
mol = Chem.MolFromSmiles(smiles)

# Assign stereochemistry using RDKit
Chem.AssignStereochemistry(mol, cleanIt=True, force=True)

# Find chiral centers and their configurations
chiral_centers = Chem.FindMolChiralCenters(mol, includeUnassigned=True)

# Print the results
for center in chiral_centers:
    index, configuration = center
    print(f"Atom index: {index}, Configuration: {configuration}")

# Helper Methods

In [ ]:
# Helper to look at predictions vs target
from math import sqrt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 12.0
plt.rcParams['figure.figsize'] = 14.0, 5.0
sns.set_theme(style='darkgrid')
def plot_predictions(df, line=True, color="PredError"):
    
    # Dataframe of the targets and predictions
    target = 'Actual Solubility'
    pred = 'Predicted Solubility'
    df_plot = pd.DataFrame({target: df['log_s'], pred: df['prediction']})
    
    # Compute Error per prediction
    if color == "PredError":
        df_plot["PredError"] = df_plot.apply(lambda x: abs(x[pred] - x[target]), axis=1)
    else:
        df_plot[color] = df[color]

    #df_plot['error'] = df_plot.apply(lambda x: abs(x[pred] - x[target]), axis=1)
    ax = df_plot.plot.scatter(x=target, y=pred, c=color, cmap='coolwarm', sharex=False)
    
    # Just a diagonal line
    if line:
        ax.axline((1, 1), slope=1, linewidth=2, c='black')
        x_pad = (df_plot[target].max() - df_plot[target].min())/10.0 
        y_pad = (df_plot[pred].max() - df_plot[pred].min())/10.0
        plt.xlim(df_plot[target].min()-x_pad, df_plot[target].max()+x_pad)
        plt.ylim(df_plot[pred].min()-y_pad, df_plot[pred].max()+y_pad)

    